### Zinc15
- Bcl2 Family inhibitor dataset by "T. Sterling, J.J. Irwin, ZINC 15–ligand discovery for everyone, J. Chem. Inf. Model. 55 (11) (2015) 2324–2337."

### Chembl
- GSK3b, JNK3 inhibitor dataset by "Multi-Objective Molecule Generation using Interpretable Substructures"
- link: [https://proceedings.mlr.press/v119/jin20b/jin20b.pdf](https://proceedings.mlr.press/v119/jin20b/jin20b.pdf)



In [5]:
import selfies as sf
import numpy as np
import pandas as pd
import string
import os
import tqdm
from transformers import AutoTokenizer, AutoModel
import re
from rdkit import Chem

In [6]:
from utils import normalize_SMILES

data_path = './data/zinc15_smiles.csv' # put your smiles data 
df = pd.read_csv(data_path)
smiles = df['smiles'].to_numpy()

norm_smiles = [normalize_SMILES(s) for s in smiles]
df['smiles'] = norm_smiles

sf_list = []
smi_list = []
for idx, smi in tqdm.tqdm(enumerate(smiles)):
    encode_sf = sf.encoder(smi)
    sf_list.append(encode_sf)

df.insert(2, 'selfies', sf_list)
df.to_csv('./data/zinc15_sample_test.csv', index=False)

1000it [00:00, 2232.39it/s]


# Make Token Dict

In [14]:
def selfies_voca(selfies_list):
    selfies_list = list(selfies_list)
    max_len = max(sf.len_selfies(s) for s in selfies_list)

    print("Max Len in Dataset : ", max_len)
    dict_voca = {}
    for src in tqdm.tqdm(selfies_list):
        alphabet = sf.get_alphabet_from_selfies([src])
        for token in alphabet:
            if token in dict_voca:
                dict_voca[token] +=1
            else:
                dict_voca[token] = 1

    sorted_dict_voca = sorted(dict_voca.items(), key=lambda x: x[1], reverse=True)
    voca = [ key for key, value in sorted_dict_voca ]
    return sorted_dict_voca, voca, max_len

def save_tokens(vocabs, filename=None):
    with open(filename, 'w') as file:
        for token in vocabs:
            file.write(f"{token}\n")


data_path = './data/zinc15_sample_test.csv'
df = pd.read_csv(data_path)
selfies = df['selfies'].to_numpy()

sorted_dict_voca, voca, max_len = selfies_voca(selfies)
voca = sorted(voca, key=lambda x: len(x), reverse=True)
save_tokens(voca, filename="./data/zinc15_selfies_tokens_test.txt")
print(voca)

Max Len in Dataset :  123


100%|█████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 20381.97it/s]

['[=Branch1]', '[#Branch1]', '[#Branch2]', '[=Branch2]', '[Branch1]', '[Branch2]', '[=Ring1]', '[=Ring2]', '[Ring1]', '[Ring2]', '[=N+1]', '[O-1]', '[N+1]', '[NH1]', '[=C]', '[=O]', '[=N]', '[#C]', '[Cl]', '[Br]', '[=S]', '[=P]', '[C]', '[N]', '[S]', '[O]', '[P]', '[F]']
